In [ ]:
from workshop_utils import *
import pandas as pd
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Global Edit Counts
This query searches only the changesets table getting basic statistics (for all users); This is effectively the same as Notebook 0, but without any changeset comment filtering. It should return information for all 1.3M contributors
```sql
SELECT
    changesets.user,
    min(created_at) AS first_edit, 
    max(created_at) AS last_edit, 
    date_diff('day', min(created_at), max(created_at)) AS lifespan,
    sum(num_changes) AS total_edits,
    count(id) AS total_changesets
FROM 
    changesets
GROUP BY 
    changesets.user 
ORDER BY lifespan DESC
```

In [ ]:
df = load_dataframe_from_s3('https://us-east-2.console.aws.amazon.com/athena/query/results/6cab4ea3-8431-4cd6-8f89-8881fa43c8b2/csv')

In [ ]:
df.set_index('user', inplace=True)
df.first_edit = df.first_edit.apply(lambda x: pd.Timestamp(x).date())
df.last_edit = df.last_edit.apply(lambda x: pd.Timestamp(x).date())

In [ ]:
df.head(2)

In [ ]:
sns.set_style('whitegrid')
ax = df.groupby('first_edit').aggregate('count')['total_edits'].plot(figsize=(14,4))
ax.set_xlabel("Date"); 
ax.set_ylabel("Number of mappers with\ntheir first edit", fontsize=16)
ax.set_title("How many users started mapping each day?", fontsize=16);

In [ ]:
ax = df.groupby('first_edit').aggregate('count')['total_edits'].cumsum().plot(figsize=(14,4))
ax.set_xlabel("Date"); 
ax.set_ylabel("Cumulative number of mappers", fontsize=16)
ax.set_title("Growth of Mappers over time", fontsize=16);

But what is that Spike at 2017?

In [ ]:
#Group by date, sort by number of users descending...
df.groupby('first_edit').aggregate('count').sort_values(by='total_edits',ascending=False).head()

In [ ]:
#Let's plot the lifespan for the userse that joined that day?
ax = df[ df.first_edit == pd.Timestamp('2017-01-26').date()][
    'lifespan'].plot(kind='hist',bins=50,figsize=(10,5))
ax.set_xlabel("Number of Days between first and last edit",fontsize=12); ax.set_ylabel("Number of mappers (log10)", fontsize=12)
ax.set_title("Histogram of mapper lifespans for mappers whose first edit was January 25, 2017", fontsize=12);

So, the majority of these users only edited on this one day. Does anyone know what caused this?

In [ ]:
ax = df['lifespan'].plot(kind='hist',bins=50,figsize=(10,5))
ax.set_yscale('log')
ax.set_xlabel("Number of Days between first and last edit"); ax.set_ylabel("Number of mappers (log10)", fontsize=12)
ax.set_title("Histogram of mapper lifespans (time between first and last edit)", fontsize=12);

Pretty standard for online communities...